In [13]:
rm(list=ls())

#########################################################################
### -- ANALÍTICA PREDICTIVA DE DATOS -- ## 
#########################################################################
### Autores: Jose Cardenas - Jose Taquia ## 
#########################################################################

#########################################################################


source("./ins_paquetes.R")

ins_paquetes('rminer','lattice','sqldf','DMwR','caret','C50','randomForest')

Warning message:
"package 'sqldf' was built under R version 3.6.3"Warning message:
"package 'RSQLite' was built under R version 3.6.3"Error: package or namespace load failed for 'RSQLite' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 namespace 'blob' 1.1.1 is being loaded, but >= 1.2.0 is required



Bellow Packages Successfully Installed:

      rminer      lattice        sqldf         DMwR        caret          C50 
        TRUE         TRUE        FALSE         TRUE         TRUE         TRUE 
randomForest 
        TRUE 


In [14]:
########### 1) LIBRERIAS A UTILIZAR ################# 

library(dplyr)
library(MLmetrics)
library(party)
library(rminer)
library(lattice)
library(mlr)
library(sqldf)
library(ggplot2)
library(DMwR)
library(caret)
library(C50)
library(rpart)
library(randomForest)

Warning message:
"package 'sqldf' was built under R version 3.6.3"Loading required package: RSQLite
Warning message:
"package 'RSQLite' was built under R version 3.6.3"Error: package or namespace load failed for 'RSQLite' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 namespace 'blob' 1.1.1 is being loaded, but >= 1.2.0 is required


ERROR: Error: package 'RSQLite' could not be loaded


In [15]:
########### 2) DATA A UTILIZAR ################# 

train <- read.csv("./datasets/german_credit_data_train.csv")

In [16]:
########### 3) TRATAMIENTO DE LA DATA ################# 

## en primer lugar ver el analisis descriptivo de la data

resumen <- data.frame(summarizeColumns(train))
resumen
write.csv(resumen,"tabla_resumen.csv")

name,type,na,mean,disp,median,mad,min,max,nlevs
Age,integer,0,35.546000,11.3754686,33.0,10.3782,19,75,0
Sex,factor,0,NA,0.3100000,NA,NA,310,690,2
Job,integer,1,1.904905,0.6533143,2.0,0.0000,0,3,0
Housing,factor,0,NA,0.2870000,NA,NA,108,713,3
Saving_accounts,factor,182,NA,NA,NA,NA,1,603,5
Checking_account,factor,394,NA,NA,NA,NA,63,274,3
Credit_amount,integer,0,3271.258000,2822.7368760,2319.5,1627.1535,250,18424,0
Duration,integer,0,20.903000,12.0588145,18.0,8.8956,4,72,0
Purpose,factor,0,NA,0.6630000,NA,NA,12,337,8
Risk,factor,0,NA,0.3000000,NA,NA,300,700,2


In [17]:
########### 4) PRIMER ANALISIS  ################# 

## generamos una copia de la data original

data_train <- train
head(data_train)

Age,Sex,Job,Housing,Saving_accounts,Checking_account,Credit_amount,Duration,Purpose,Risk
67,male,2,own,NA,little,1169,6,radio/TV,good
22,female,2,own,little,moderate,5951,48,radio/TV,bad
49,male,1,own,little,NA,2096,12,education,good
45,male,2,free,little,little,7882,42,furniture/equipment,good
53,male,2,free,little,little,4870,24,car,bad
35,male,NA,free,,NA,9055,36,education,good


In [29]:
## imputar en primer lugar la data

#detach("package:e1071", unload = TRUE)

data_train <- mlr::impute(train, classes = list(factor = imputeMode(), 
                                                  integer = imputeMode(),
                                                  numeric = imputeMedian()))
data_train <- data_train$data[,1:ncol(train)]

In [30]:
## luego dar un orden a las variables

summary(data_train)
cuantis <- data_train %>% select(Age,Credit_amount,Duration)
cualis <- data_train %>% select(-Age,-Credit_amount,-Duration)

data_train <- cbind(cuantis,cualis)

      Age            Sex           Job        Housing      Saving_accounts
 Min.   :19.00   female:310   Min.   :0.000   free:108             :  1   
 1st Qu.:27.00   male  :690   1st Qu.:2.000   own :713   little    :785   
 Median :33.00                Median :2.000   rent:179   moderate  :103   
 Mean   :35.55                Mean   :1.905              quite rich: 63   
 3rd Qu.:42.00                3rd Qu.:2.000              rich      : 48   
 Max.   :75.00                Max.   :3.000                               
                                                                          
 Checking_account Credit_amount      Duration                   Purpose   
 little  :668     Min.   :  250   Min.   : 4.0   car                :337  
 moderate:269     1st Qu.: 1366   1st Qu.:12.0   radio/TV           :280  
 rich    : 63     Median : 2320   Median :18.0   furniture/equipment:181  
                  Mean   : 3271   Mean   :20.9   business           : 97  
                  3rd Qu.

In [31]:
# recodificacion manual
data_train$Sex <- ifelse(data_train$Sex=='male',2,1)
data_train$Housing <- ifelse(data_train$Housing == 'own',1,
                             ifelse(data_train$Housing =='rent', 2,3))
data_train$Saving_accounts <- ifelse(data_train$Saving_accounts == 'little',1,
                                     ifelse(data_train$Saving_accounts == 'moderate',2,
                                            ifelse(data_train$Saving_accounts == 'rich',3,4)))
data_train$Checking_account <- ifelse(data_train$Checking_account == 'little',1,
                                      ifelse(data_train$Checking_account == 'moderate',2,3))

data_train$Purpose <- ifelse(data_train$Purpose == 'car',1,
                             ifelse(data_train$Purpose == 'furniture/equipment',2,
                                    ifelse(data_train$Purpose == 'radio/TV',3,
                                           ifelse(data_train$Purpose == 'domestic appliances',4,
                                                  ifelse(data_train$Purpose == 'repairs',5,
                                                         ifelse(data_train$Purpose == 'education',6,
                                                                ifelse(data_train$Purpose == 'business',7,8)))))))

data_train$Risk <- ifelse(data_train$Risk=='bad',1,0)

In [32]:
# categorizando a factor

data_train[,(ncol(cuantis)+1):ncol(data_train)] <- lapply(data_train[,(ncol(cuantis)+1):ncol(data_train)],as.factor)
#lapply : Apply a Function over a List or Vector
summary(data_train)

data_train<-na.omit(data_train)
summary(data_train)

      Age        Credit_amount      Duration    Sex     Job     Housing
 Min.   :19.00   Min.   :  250   Min.   : 4.0   1:310   0: 22   1:713  
 1st Qu.:27.00   1st Qu.: 1366   1st Qu.:12.0   2:690   1:199   2:179  
 Median :33.00   Median : 2320   Median :18.0           2:631   3:108  
 Mean   :35.55   Mean   : 3271   Mean   :20.9           3:148          
 3rd Qu.:42.00   3rd Qu.: 3972   3rd Qu.:24.0                          
 Max.   :75.00   Max.   :18424   Max.   :72.0                          
                                                                       
 Saving_accounts Checking_account    Purpose    Risk   
 1:785           1:668            1      :337   0:700  
 2:103           2:269            3      :280   1:300  
 3: 48           3: 63            2      :181          
 4: 64                            7      : 97          
                                  6      : 59          
                                  5      : 22          
                                

      Age        Credit_amount      Duration    Sex     Job     Housing
 Min.   :19.00   Min.   :  250   Min.   : 4.0   1:310   0: 22   1:713  
 1st Qu.:27.00   1st Qu.: 1366   1st Qu.:12.0   2:690   1:199   2:179  
 Median :33.00   Median : 2320   Median :18.0           2:631   3:108  
 Mean   :35.55   Mean   : 3271   Mean   :20.9           3:148          
 3rd Qu.:42.00   3rd Qu.: 3972   3rd Qu.:24.0                          
 Max.   :75.00   Max.   :18424   Max.   :72.0                          
                                                                       
 Saving_accounts Checking_account    Purpose    Risk   
 1:785           1:668            1      :337   0:700  
 2:103           2:269            3      :280   1:300  
 3: 48           3: 63            2      :181          
 4: 64                            7      : 97          
                                  6      : 59          
                                  5      : 22          
                                

In [33]:
######### 5) PARTICION MUESTRAL  #################

## Particionando la Data

set.seed(1234)
sample <- createDataPartition(data_train$Risk, p = .70,list = FALSE,times = 1)

data.train <- data_train[ sample,]
data.test <- data_train[-sample,]

In [37]:
######### 6) MODELADO #################

# modelo 1.- Random forest

modelo1 <- randomForest::randomForest( Risk~.,data = data.train,   # Datos a entrenar 
                         ntree=50,           # N?mero de ?rboles
                         mtry = 3,            # Cantidad de variables
                         importance = TRUE,   # Determina la importancia de las variables
                         replace=T)

# probabilidades
proba1 <- predict(modelo1, newdata=data.test,type="prob")
proba1 <- proba1[,2]

In [38]:
# indicadores para probabilidades
GINI_1     <- Gini(proba1,   as.numeric(as.character(data.test$Risk)))
ks_1       <- KS_Stat(proba1,as.numeric(as.character(data.test$Risk)))
LogLoss_1  <- LogLoss(proba1,as.numeric(as.character(data.test$Risk)))

# Calcular los valores predichos
PRED <- predict(modelo1, newdata=data.test,type="response")

In [39]:
# Calcular la matriz de confusion
tabla <- confusionMatrix(PRED,data.test$Risk,positive = "1")
tabla

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 175  64
         1  35  26
                                         
               Accuracy : 0.67           
                 95% CI : (0.6136, 0.723)
    No Information Rate : 0.7            
    P-Value [Acc > NIR] : 0.883683       
                                         
                  Kappa : 0.1346         
                                         
 Mcnemar's Test P-Value : 0.004891       
                                         
            Sensitivity : 0.28889        
            Specificity : 0.83333        
         Pos Pred Value : 0.42623        
         Neg Pred Value : 0.73222        
             Prevalence : 0.30000        
         Detection Rate : 0.08667        
   Detection Prevalence : 0.20333        
      Balanced Accuracy : 0.56111        
                                         
       'Positive' Class : 1              
                                         

In [40]:
# sensibilidad
Sensitivity_1 <- as.numeric(tabla$byClass[1])

In [41]:
# Precision
Accuracy_1 <- tabla$overall[1]
Accuracy_1

Accuracy 
    0.67